# Build NSF COA Co-Author List from DOI

NSF requires a list of collaborators and co-authors as an accompanying document for proposals.  This is onerous, especially if you have papers with many, many co-authors.  This notebook utilizes DOI bibliographic records and ORCID to create this list.

All that is required is a list of DOI and a list of collaborators.

In [28]:
import requests
import json

from orcid import Orcid

In [42]:
def crossref_url(doi, format="json"):
    """Retrieves a DOI record from crossref"""
    return f"http://api.crossref.org/works/{doi}/transform/application/{format}"

def get_doi_record(doi, format="json"):
    url = crossref_url(doi, format=format)
    r = requests.get(url)
    return r.content

class Author:

    def __init__(self, author_info):
        """Initializes Author
        
        author_info : dict or author information from doi json
        """
        self.orcid = author_info["ORCID"]
        self.given = author_info["given"]
        self.family = author_info["family"]
        self.sequence = author_info["sequence"]
        self.affiliation = [af["name"] for af in author_info["affiliation"]]
        self.email = author_info.get("email", "")

    def to_list(self):
        return [f"{self.family}, {self.given}", f"{self.affiliation[0]}", f"{self.email}"]

    def email_from_orcid(self):
        if self.orcid != "":
            oid = self.orcid.split("/")[-1]
            self.email = Orcid(oid).email

In [13]:
doi = "10.1002/ece3.11341"
record = json.loads(get_doi_record(doi))

In [30]:
author = Author(record["author"][0])

In [36]:
author.to_list()

['Lowndes, Julia Stewart',
 'University of California, Santa Barbara  Santa Barbara California USA',
 '[]']

In [37]:
author.email_from_orcid()

In [38]:
author.to_list()

['Lowndes, Julia Stewart',
 'University of California, Santa Barbara  Santa Barbara California USA',
 '[]']

In [48]:
authors = [Author(a) for a in record["author"]]
authors = [a.to_list() for a in authors if a.family != "Barrett"]

In [52]:
["C:"] + authors[0]

['C:',
 'Lowndes, Julia Stewart',
 'University of California, Santa Barbara  Santa Barbara California USA',
 '']

In [50]:
import csv
with open('some.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["name", "affiliation", "email"])
    writer.writerows(authors)
